## Import the Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import seaborn as sns
import geoplot as gplt
import geoplot.crs as gcrs
import folium
import matplotlib.ticker as ticker
import scipy.stats
import webbrowser
from scipy.stats import shapiro
import statsmodels.api as sm
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from sklearn.preprocessing import MinMaxScaler
import matplotlib
from IPython.display import display, HTML, Image
import json
from scipy import stats
%matplotlib inline
import dataframe_image as dfi
import mgwr
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colormap
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point
import pysal
from pysal.model import spreg
import textwrap
import matplotlib.colors as colors#
import statistics

## Read in the Data

In [ ]:
# read in the Number of Households Data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Number of Households Data.xlsx"
NumHouse = pd.read_excel(file_path)

# read in the Population density Data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Population Density Data.xlsx"
PopDensity = pd.read_excel(file_path)

# read in the RUC classification Data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Rural Urban Classification.xlsx"
RUC11 = pd.read_excel(file_path)

# read in the LSOA11 - LSOA21 Lookup data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/LSOA (2011) to LSOA (2021).csv"
LSOA11_21 = pd.read_csv(file_path)

# read in the LSOA21 - MSOA21 Lookup data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/LSOA 2021 to MSOA 2021.xlsx"
LSOA21_MSOA21 = pd.read_excel(file_path)

# read in the population age data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Oxfordshire Age by Single Year.xlsx"
Ox_PopAge = pd.read_excel(file_path)

# read in the household composition data
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Household Composition Data (2).xlsx"
Ox_Household_Comp = pd.read_excel(file_path)

## Group the population age data to get the average age per MSOA

In [ ]:
def weighted_average(MSOA21CD):
    return round((MSOA21CD['Number']*MSOA21CD['Age (101 categories) Code']).sum()/MSOA21CD['Number'].sum())

In [ ]:
MSOA_Avg_Age = pd.DataFrame(Ox_PopAge.groupby(['MSOA21CD','MSOA21NM']).apply(weighted_average).reset_index())
MSOA_Avg_Age = MSOA_Avg_Age.rename(columns = {0:'Average Age'})

### Merge the population age data with the LSOA-MSOA 21 Lookup

In [ ]:
Ox_Avg_Age = MSOA_Avg_Age.merge(LSOA21_MSOA21[['LSOA21CD','LSOA21NM','MSOA21CD']], on='MSOA21CD',how='inner')

In [ ]:
Ox_Avg_Age = Ox_Avg_Age.drop_duplicates()

In [ ]:
Ox_Avg_Age = Ox_Avg_Age.reindex(columns = ['MSOA21CD','MSOA21NM','LSOA21CD','LSOA21NM','Average Age'])

## Merge the LSOA lookup table with the demographics data

In [ ]:
Ox_dem = RUC11.merge(LSOA11_21[['LSOA11CD','LSOA21CD','LSOA21NM']], on='LSOA11CD',how='inner')
Ox_dem = Ox_dem.merge(PopDensity[['LSOA21CD','PopDensity']], on='LSOA21CD', how='inner')
Ox_dem = Ox_dem.merge(NumHouse[['LSOA21CD','NumHouse']], on='LSOA21CD',how='inner')

In [ ]:
desired_order = ['LSOA11CD','LSOA11NM','LSOA21CD','LSOA21NM','RUC11CD','RUC11','PopDensity','NumHouse']
Ox_dem = Ox_dem.reindex(columns=desired_order)

In [ ]:
Ox_dem = Ox_dem.merge(Ox_Avg_Age[['LSOA21CD','Average Age']], on='LSOA21CD', how='inner')

In [ ]:
Ox_Household_Comp.columns

In [ ]:
Ox_dem = Ox_dem.merge(Ox_Household_Comp[['LSOA21CD','One-person household: Aged 66 years and over',
       'One-person household: Other',
       'Single family household: All aged 66 years and over',
       'Single family household: Married or civil partnership couple: No children',
       'Single family household: Married or civil partnership couple: Dependent children',
       'Single family household: Married or civil partnership couple: All children non-dependent',
       'Single family household: Cohabiting couple family: No children',
       'Single family household: Cohabiting couple family: With dependent children',
       'Single family household: Cohabiting couple family: All children non-dependent',
       'Single family household: Lone parent family: With dependent children',
       'Single family household: Lone parent family: All children non-dependent',
       'Single family household: Other single family household: Other family composition',
       'Other household types: With dependent children',
       'Other household types: Other, including all full-time students and all aged 66 years and over']],
                     on='LSOA21CD',how='inner')

## Add rank columns for each of population density and number of households

In [ ]:
Ox_dem['Rank_PopDensity'] = pd.qcut(Ox_dem['PopDensity'],10,labels=np.arange(1,11))
Ox_dem['Rank_NumHouse'] = pd.qcut(Ox_dem['NumHouse'],10,labels=np.arange(1,11))

## Export the file to an Excel workbook

In [ ]:
Ox_dem.to_excel("C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Oxon Demographics (RUC, Household, Pop Density).xlsx")